# Tourism Demand Forecasting with Spark


## Feng Li

### Guanghua School of Management
### Peking University


### [feng.li@gsm.pku.edu.cn](feng.li@gsm.pku.edu.cn)
### Course home page: [https://feng.li/bdcf](https://feng.li/bdcf)

In [ ]:
import os, sys # Ensure All environment variables are properly set 
# os.environ["JAVA_HOME"] = os.path.dirname(sys.executable)
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark.sql import SparkSession # build Spark Session
spark = SparkSession.builder \
    .config("spark.ui.enabled", "false") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.cores.max", "32") \
    .config("spark.driver.memory", "30g") \
    .config("spark.sql.shuffle.partitions", "96") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.5") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "4") \
    .config("spark.dynamicAllocation.maxExecutors", "8") \
    .appName("Spark Forecasting").getOrCreate()
spark

## Summary of the Data Structure

- Tourism Data:

    - 228 rows (time series data).
    - 556 columns (date + 555 individual series representing regions/categories).

- Summing Matrix:

    - 555 rows (each corresponding to a region/category).
    - 305 columns (Parent_Group + 304 child region/category columns).
    - The first column (Parent_Group) correctly defines hierarchical aggregation.

In [2]:
from pyspark.sql.functions import col, max as spark_max
from pyspark.sql.types import DateType


# Load the tourism dataset (update the path accordingly)
file_path = "../data/tourism/tourism.csv"  # Replace with actual file path
sdf = spark.read.csv(file_path, header=True, inferSchema=True)

# Ensure 'date' column is in proper DateType format
sdf = sdf.withColumn("date", col("date").cast(DateType()))

# Convert wide format to long format
columns_to_melt = [c for c in sdf.columns if c != "date"]
sdf_long = sdf.selectExpr(
    "date",
    f"stack({len(columns_to_melt)}, " + ", ".join([f"'{c}', {c}" for c in columns_to_melt]) + ") as (Region_Category, Visitors)"
)

# Find the maximum date in the dataset
max_date = sdf_long.select(spark_max("date")).collect()[0][0]

# Define the threshold date for splitting (last 12 months for testing)
split_date = max_date.replace(year=max_date.year - 1)

# Split into training and testing datasets
train_sdf = sdf_long.filter(col("date") < split_date)
test_sdf = sdf_long.filter(col("date") >= split_date)

# Show results
train_sdf.show(5)
test_sdf.show(5)

# Save if needed
train_sdf.write.csv(os.path.expanduser("~/train_data.csv"), header=True, mode="overwrite")
test_sdf.write.csv(os.path.expanduser("~/test_data.csv"), header=True, mode="overwrite")

25/03/18 20:26:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---------------+------------------+
|      date|Region_Category|          Visitors|
+----------+---------------+------------------+
|1998-01-01|       TotalAll|45151.071280099975|
|1998-01-01|           AAll|17515.502379600006|
|1998-01-01|           BAll|10393.618015699998|
|1998-01-01|           CAll| 8633.359046599999|
|1998-01-01|           DAll|3504.3133462000005|
+----------+---------------+------------------+
only showing top 5 rows

+----------+---------------+------------------+
|      date|Region_Category|          Visitors|
+----------+---------------+------------------+
|2015-12-01|       TotalAll|24982.024449599998|
|2015-12-01|           AAll|      7166.1237555|
|2015-12-01|           BAll|      5340.1512778|
|2015-12-01|           CAll| 5621.323498100002|
|2015-12-01|           DAll|1871.7627924999997|
+----------+---------------+------------------+
only showing top 5 rows



## Convert wide format to long format

- The orignial data are in long format

- We convert it in long format

In [3]:
# the original wide format
sdf.toPandas()

,date,TotalAll,AAll,BAll,CAll,DAll,EAll,FAll,GAll,AAAll,...,GBBBus,GBBOth,GBCHol,GBCVis,GBCBus,GBCOth,GBDHol,GBDVis,GBDBus,GBDOth
0,1998-01-01,45151.071280,17515.502380,10393.618016,8633.359047,3504.313346,3121.619189,1850.735773,131.923529,4977.209611,...,0.000000,0.000000,7.536223,0.000000,1.628948,0.000000,0.811856,0.000000,9.478051,0.0
1,1998-02-01,17294.699551,5880.367918,3855.647839,3580.051065,1321.257992,1826.610676,757.079744,73.684316,1937.229611,...,1.045797,0.000000,0.000000,0.000000,5.296459,0.000000,0.522899,0.000000,0.000000,0.0
2,1998-03-01,20725.114184,7086.444392,4353.379282,4717.676663,1521.950007,1868.381530,900.796622,276.485688,2117.671851,...,0.000000,0.000000,2.945006,1.425324,9.924744,3.100121,0.000000,0.000000,0.000000,0.0
3,1998-04-01,25388.612353,10530.639348,5115.865530,4924.575204,1813.439177,1952.612465,801.444140,250.036488,2615.957465,...,11.461824,0.000000,26.419176,13.690603,2.312088,0.000000,0.000000,10.958005,2.312088,0.0
4,1998-05-01,20330.035211,7430.373559,3820.666426,4219.283647,1375.082095,2616.965317,551.377058,316.287109,2393.145511,...,0.000000,0.000000,23.789282,67.846207,1.282767,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2016-08-01,24100.446632,7227.612581,4000.422405,7536.939346,1230.321700,2868.505157,355.515713,881.129730,2305.649511,...,7.874448,0.000000,49.899947,8.300225,17.633478,8.218562,4.121631,0.408894,1.625820,0.0
224,2016-09-01,24800.033759,6778.363226,4132.168965,7123.112802,1632.732153,3327.064770,525.302974,1281.288869,2061.246613,...,0.421996,0.000000,80.058887,37.306013,109.024164,46.447153,11.661140,3.557735,4.448534,0.0
225,2016-10-01,30039.106985,8592.998250,5719.297913,8759.191781,1900.476487,3704.651986,895.180382,467.310186,2267.174784,...,35.375093,0.273247,52.156131,2.093902,50.283538,3.319366,0.754941,0.000000,0.000000,0.0
226,2016-11-01,27320.918908,8663.240960,5165.403172,6804.359328,1543.299435,3698.431886,852.313563,593.870565,2786.280116,...,10.723090,39.292496,8.344998,7.697995,43.270319,0.000000,0.000000,0.000000,5.180648,0.0


In [4]:
# the long format
sdf_long.show()

+----------+---------------+------------------+
|      date|Region_Category|          Visitors|
+----------+---------------+------------------+
|1998-01-01|       TotalAll|45151.071280099975|
|1998-01-01|           AAll|17515.502379600006|
|1998-01-01|           BAll|10393.618015699998|
|1998-01-01|           CAll| 8633.359046599999|
|1998-01-01|           DAll|3504.3133462000005|
|1998-01-01|           EAll|      3121.6191894|
|1998-01-01|           FAll|1850.7357734999998|
|1998-01-01|           GAll|131.92352909999997|
|1998-01-01|          AAAll|      4977.2096105|
|1998-01-01|          ABAll| 5322.738721600001|
|1998-01-01|          ACAll|3569.6213724000004|
|1998-01-01|          ADAll|      1472.9706096|
|1998-01-01|          AEAll|      1560.5142545|
|1998-01-01|          AFAll|        612.447811|
|1998-01-01|          BAAll|       3854.672582|
|1998-01-01|          BBAll|1653.9957826000002|
|1998-01-01|          BCAll|      2138.7473162|
|1998-01-01|          BDAll|      1395.3

## The long format in Spark processing 

### Efficient Parallelization in Spark

- Spark natively processes data in a row-wise manner, so applying a Pandas UDF on each column separately is inefficient.
- By reshaping into long format, each (Region_Category, Visitors) pair becomes a separate row, allowing Spark to parallelize operations better.

### Scalability for Large Datasets

- If you have hundreds of columns (as in our dataset), keeping it in wide format means:
    - The UDF would need to handle many columns at once.
    - Each worker in Spark must load all columns, increasing memory pressure.
- In long format, each time series is processed independently, allowing Spark to distribute tasks across multiple nodes efficiently.

### Simpler UDF Application

- Spark natively groups and applies UDFs on rows rather than columns.
- Instead of manually iterating over columns in Python (which is slow and not parallelized), Spark can efficiently apply the UDF per region.

In [5]:
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType, DateType
import pandas as pd
from pandas.tseries.offsets import MonthBegin
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Define schema for the forecast output
forecast_schema = StructType([
    StructField("date", DateType(), False),
    StructField("Region_Category", StringType(), False),
    StructField("Forecast", DoubleType(), False)
])


def ets_forecast(pdf):
    """Fits an ETS model for a single region and forecasts 12 months ahead."""
    region = pdf["Region_Category"].iloc[0]  # Extract region name
    pdf = pdf.sort_values("date")  # Ensure time series is sorted

    try:
        # Drop missing values
        ts = pdf["Visitors"].dropna()

        if len(ts) >= 24:  # Ensure at least 24 observations
            model = ExponentialSmoothing(ts, trend="add", seasonal="add", seasonal_periods=12)
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=12)
        else:
            forecast = [None] * 12  # Not enough data
    except:
        forecast = [None] * 12  # Handle errors

    # Adjust forecast dates to start of the month
    last_date = pdf["date"].max()
    future_dates = pd.date_range(start=last_date, periods=12, freq="ME") + MonthBegin(1)

    # Return results as a DataFrame
    return pd.DataFrame({"date": future_dates, "Region_Category": region, "Forecast": forecast})

# Apply the ETS model in parallel using applyInPandas
forecast_sdf = train_sdf.groupBy("Region_Category").applyInPandas(ets_forecast, schema=forecast_schema)

# Show forecasted results
forecast_sdf.show()

# Save forecasts if needed
forecast_sdf.write.csv(os.path.expanduser("~/ets_forecasts.csv"), header=True, mode="overwrite")


/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+----------+---------------+------------------+
|      date|Region_Category|          Forecast|
+----------+---------------+------------------+
|2015-12-01|         AAAAll| 2058.838101212888|
|2016-01-01|         AAAAll|3162.9085270260384|
|2016-02-01|         AAAAll|1744.1909768938476|
|2016-03-01|         AAAAll|2059.3010229302345|
|2016-04-01|         AAAAll|  2060.36170915585|
|2016-05-01|         AAAAll| 1972.482680954841|
|2016-06-01|         AAAAll| 1846.108381522378|
|2016-07-01|         AAAAll| 2151.959971338432|
|2016-08-01|         AAAAll| 1872.768734964083|
|2016-09-01|         AAAAll|2014.0112033543805|
|2016-10-01|         AAAAll|2296.4055573391643|
|2016-11-01|         AAAAll| 2043.693618904705|
|2015-12-01|         AAABus|455.55263433165527|
|2016-01-01|         AAABus| 296.1898590727801|
|2016-02-01|         AAABus| 453.3714726155002|
|2016-03-01|         AAABus|  525.339287022726|
|2016-04-01|         AAABus| 459.2040763240983|
|2016-05-01|         AAABus| 554.7602408

/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [6]:
from pyspark.sql.functions import abs, mean, col

# Join forecasted results with actual test data on date and Region_Category
evaluation_sdf = forecast_sdf.join(
    test_sdf, on=["date", "Region_Category"], how="inner"
).withColumn("APE", abs((col("Forecast") - col("Visitors")) / col("Visitors")))

# Compute Mean Absolute Percentage Error (MAPE) for each Region_Category
mape_sdf = evaluation_sdf.groupBy("Region_Category").agg(mean("APE").alias("MAPE"))

# Show MAPE results
mape_sdf.show()
# Save MAPE results if needed
# mape_sdf.write.csv(os.path.expanduser("~/mape_results.csv"), header=True, mode="overwrite")

# Compute overall mean MAPE across all Region_Category
overall_mape = mape_sdf.agg(mean("MAPE").alias("Overall_MAPE"))

# Show the result
overall_mape.show()

/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+---------------+-------------------+
|Region_Category|               MAPE|
+---------------+-------------------+
|         BCBOth| 1.6313505417732714|
|          BCHol|0.24248111440660092|
|         BDEAll|0.49900559071075934|
|         CBDAll| 0.2463100287348122|
|         CCBAll|0.13439393461339585|
|          CCOth|  0.958426283463088|
|         DCCAll| 0.7927858168878638|
|         DDBHol|0.48647432685897135|
|         EABVis|0.20529851654215334|
|         FBAVis| 1.1987426872820532|
|         ADBAll| 0.2475814713509836|
|         BDFAll| 0.7134931417274598|
|         CBCHol|0.47280752400465315|
|         FAAHol|0.22072859753473625|
|         GABVis|0.45942041750965995|
|         GBCAll| 0.3450829531649569|
|          AEHol|0.17882207692308452|
|         BDBAll| 0.7177775564201326|
|         BDCBus| 1.0623031471971673|
|         BEGAll|0.48618998882496967|
+---------------+-------------------+
only showing top 20 rows



/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+------------------+
|      Overall_MAPE|
+------------------+
|0.8896466480629734|
+------------------+



In [9]:
from pyspark.sql.functions import col

# Load the summing matrix file
summing_matrix_path = "../data/tourism/agg_mat.csv"  # Update with actual path

# Load the summing matrix file (skip the first column)
summing_sdf = spark.read.csv(summing_matrix_path, header=True, inferSchema=True)

# Show the cleaned summing matrix
summing_sdf.toPandas()

,Parent_Group,AAAHol,AAAVis,AAABus,AAAOth,AABHol,AABVis,AABBus,AABOth,ABAHol,...,GBBBus,GBBOth,GBCHol,GBCVis,GBCBus,GBCOth,GBDHol,GBDVis,GBDBus,GBDOth
0,TotalAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,GBCOth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
551,GBDHol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
552,GBDVis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
553,GBDBus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
from pyspark.sql.functions import expr

# Convert from wide format to long format (Region_Category, Parent_Group, Weight)
summing_sdf_long = summing_sdf.selectExpr(
    "Parent_Group",
    "stack(" + str(len(summing_sdf.columns) - 1) + ", " +
    ", ".join([f"'{col}', {col}" for col in summing_sdf.columns if col != "Parent_Group"]) +
    ") as (Region_Category, Weight)"
)

# Show the reshaped summing matrix
summing_sdf_long.show()

+------------+---------------+------+
|Parent_Group|Region_Category|Weight|
+------------+---------------+------+
|    TotalAll|         AAAHol|   1.0|
|    TotalAll|         AAAVis|   1.0|
|    TotalAll|         AAABus|   1.0|
|    TotalAll|         AAAOth|   1.0|
|    TotalAll|         AABHol|   1.0|
|    TotalAll|         AABVis|   1.0|
|    TotalAll|         AABBus|   1.0|
|    TotalAll|         AABOth|   1.0|
|    TotalAll|         ABAHol|   1.0|
|    TotalAll|         ABAVis|   1.0|
|    TotalAll|         ABABus|   1.0|
|    TotalAll|         ABAOth|   1.0|
|    TotalAll|         ABBHol|   1.0|
|    TotalAll|         ABBVis|   1.0|
|    TotalAll|         ABBBus|   1.0|
|    TotalAll|         ABBOth|   1.0|
|    TotalAll|         ACAHol|   1.0|
|    TotalAll|         ACAVis|   1.0|
|    TotalAll|         ACABus|   1.0|
|    TotalAll|         ACAOth|   1.0|
+------------+---------------+------+
only showing top 20 rows



In [12]:
# Bottom up
from pyspark.sql.functions import sum as spark_sum, col
# Join forecast data with transformed summing matrix
forecast_with_hierarchy_sdf = forecast_sdf.join(
    summing_sdf_long, on="Region_Category", how="inner"
)

# Aggregate forecasts according to the summing matrix
reconciled_forecast_sdf = forecast_with_hierarchy_sdf.groupBy("date").agg(
    spark_sum(col("Forecast") * col("Weight")).alias("Reconciled_Forecast")
)

# Show reconciled forecasts
reconciled_forecast_sdf.show()

# Save reconciled forecasts if needed
# reconciled_forecast_sdf.write.csv(os.path.expanduser("~/reconciled_forecast.csv"), header=True, mode="overwrite")

/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+----------+-------------------+
|      date|Reconciled_Forecast|
+----------+-------------------+
|2016-07-01| 217470.62241978827|
|2015-12-01|   189559.905486296|
|2016-08-01|  195853.8446184591|
|2016-09-01| 203235.25808426825|
|2016-10-01| 229027.16509088216|
|2016-04-01| 226773.57106515038|
|2016-01-01|  368077.6543055778|
|2016-02-01|  166785.6990820566|
|2016-06-01|  179587.9894170942|
|2016-11-01| 196835.93132411691|
|2016-05-01| 180658.18326216078|
|2016-03-01| 191255.45206051113|
+----------+-------------------+



In [18]:
# Map test data to Parent_Group using summing matrix
test_with_hierarchy_sdf = test_sdf.join(
    summing_sdf_long, on="Region_Category", how="inner"
)

# Aggregate forecasts at Parent_Group level
reconciled_forecast_sdf = forecast_with_hierarchy_sdf.groupBy("date", "Parent_Group").agg(
    spark_sum(col("Forecast") * col("Weight")).alias("Reconciled_Forecast")
)

# Merge test data with reconciled forecasts
evaluation_sdf = test_grouped_sdf.join(
    reconciled_forecast_sdf, on=["date", "Parent_Group"], how="inner"
)

evaluation_sdf.show()

/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+----------+------------+------------------+-------------------+
|      date|Parent_Group|   Actual_Visitors|Reconciled_Forecast|
+----------+------------+------------------+-------------------+
|2015-12-01|      DBBHol|24982.024449599998|  93.30359324977445|
|2015-12-01|      BEDBus|24982.024449599998|  9.587146785063556|
|2015-12-01|      BEAHol|24982.024449599998|  5.495956927738096|
|2015-12-01|      ADBBus|24982.024449599998|  33.48580576600097|
|2015-12-01|      ADBHol|24982.024449599998|  67.11613382657367|
|2015-12-01|       FCVis|24982.024449599998|  49.72621582615672|
|2015-12-01|       EABus|24982.024449599998|  337.9295021569872|
|2015-12-01|       ACBus|24982.024449599998|  72.84836998814711|
|2015-12-01|      GACAll|24982.024449599998| 25.191637997601127|
|2016-01-01|      DCBHol|45625.487797300026|  122.5716884443587|
|2016-01-01|      BBAHol|45625.487797300026| 1215.3210703251198|
|2016-01-01|      AFAHol|45625.487797300026| 231.28278439478635|
|2016-01-01|      ABAVis|

In [23]:
from pyspark.sql.functions import abs, mean

# Compute Absolute Percentage Error (APE)
evaluation_sdf = evaluation_sdf.withColumn(
    "APE", abs((col("Reconciled_Forecast") - col("Actual_Visitors")) / col("Actual_Visitors"))
)

# Compute MAPE for each Parent_Group
mape_bu_sdf = evaluation_sdf.groupBy("Parent_Group").agg(mean("APE").alias("MAPE"))

# Show MAPE results
mape_bu_sdf.show()

# Compute overall mean MAPE across all Region_Category
overall_mape_bu = mape_bu_sdf.agg(mean("MAPE").alias("Overall_MAPE"))

# Show the result
overall_mape_bu.show()

# Save results if needed
# mape_sdf.write.csv(os.path.expanduser("~/mape_bottom_up.csv"), header=True, mode="overwrite")


/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+------------+------------------+
|Parent_Group|              MAPE|
+------------+------------------+
|      CBDAll|0.9918551321816805|
|       BCHol|0.9857934130236617|
|      BCBOth|0.9998273964952409|
|      DDBHol|0.9965713950479036|
|      CCBAll|0.9872982701563636|
|       CCOth|0.9971415937249763|
|      DCCAll|0.9987860340371489|
|      BDEAll|0.9986953765843504|
|      FBAVis|0.9996472874910226|
|      EABVis|0.9849344998635591|
|      GABVis|0.9997748506792165|
|      ADBAll|0.9903498750139902|
|      FAAHol|0.9929209612350367|
|      BDFAll|0.9989229459704517|
|      CBCHol| 0.996186434565634|
|      GBCAll|0.9974573023722869|
|      CDBHol| 0.998463935084155|
|      BEGAll|0.9955394793307324|
|       DABus|0.9938826685744151|
|      DAAVis|0.9899752829805119|
+------------+------------------+
only showing top 20 rows



/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/nfs-share/home/2406184221/.local/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


+------------------+
|      Overall_MAPE|
+------------------+
|0.9860567355464035|
+------------------+



## Lab

- Why bottom-up method is not as good as raw forecast?
- Use some other forecasting method other than ETS for the reconciliation